In [20]:
import numpy as np
import pandas as pd
from scipy import stats

Для 61 большого города в Англии и Уэльсе известны средняя годовая смертность на 100000 населения (по данным 1958–1964) и концентрация кальция в питьевой воде (в частях на маллион). Чем выше концентрация кальция, тем жёстче вода. Города дополнительно поделены на северные и южные.
Есть ли связь между жёсткостью воды и средней годовой смертностью? Посчитайте значение коэффициента корреляции Пирсона между этими признаками, округлите его до четырёх знаков после десятичной точки.

In [3]:
water = pd.read_csv('water.txt', sep='\t')

In [4]:
water.head()

,location,town,mortality,hardness
0,South,Bath,1247,105
1,North,Birkenhead,1668,17
2,South,Birmingham,1466,5
3,North,Blackburn,1800,14
4,North,Blackpool,1609,18


In [12]:
water[['hardness', 'mortality']].corr(method='pearson')

,hardness,mortality
hardness,1.000000,-0.654849
mortality,-0.654849,1.000000


В предыдущей задаче посчитайте значение коэффициента корреляции Спирмена между средней годовой смертностью и жёсткостью воды. Округлите до четырёх знаков после десятичной точки.

In [14]:
water[['hardness', 'mortality']].corr(method='spearman')

,hardness,mortality
hardness,1.000000,-0.631665
mortality,-0.631665,1.000000


Сохраняется ли связь между признаками, если разбить выборку на северные и южные города? Посчитайте значения корреляции Пирсона между средней годовой смертностью и жёсткостью воды в каждой из двух подвыборок, введите наименьшее по модулю из двух значений, округлив его до четырёх знаков после десятичной точки.

In [15]:
water[water['location']=='South'][['hardness', 'mortality']].corr(method='pearson')

,hardness,mortality
hardness,1.000000,-0.602153
mortality,-0.602153,1.000000


In [16]:
water[water['location']=='North'][['hardness', 'mortality']].corr(method='pearson')

,hardness,mortality
hardness,1.000000,-0.368598
mortality,-0.368598,1.000000


Среди респондентов General Social Survey 2014 года хотя бы раз в месяц проводят вечер в баре 203 женщины и 239 мужчин; реже, чем раз в месяц, это делают 718 женщин и 515 мужчин.

Посчитайте значение коэффициента корреляции Мэтьюса между полом и частотой похода в бары. Округлите значение до трёх знаков после десятичной точки

In [17]:
def met(a, b, c, d):
    return (a*d - b*c)/np.sqrt((a + b)*(a + c)*(b + d)*(c + d)) 

In [18]:
met(203, 718, 239, 515)

-0.10900237458678963

В предыдущей задаче проверьте, значимо ли коэффициент корреляции Мэтьюса отличается от нуля. Посчитайте достигаемый уровень значимости; используйте функцию scipy.stats.chi2_contingency. Введите номер первой значащей цифры (например, если вы получили $5.5\times10^{-8}$, нужно ввести 8).

In [46]:
stats.chi2_contingency([[203, 239], [718, 515]])[1]

1.0558987006638725e-05

В предыдущей задаче давайте попробуем ответить на немного другой вопрос: отличаются ли доля мужчин и доля женщин, относительно часто проводящих вечера в баре? Постройте 95% доверительный интервал для разности долей, вычитая долю женщин из доли мужчин. Чему равна его нижняя граница? Округлите до четырёх знаков после десятичной точки.



In [24]:
n1 = 203+718
n2 = 239+515
    
p1 = 203 / n1
p2 = 239 / n2 

P = float(p1*n1 + p2*n2) / (n1 + n2)
Z = (p1 - p2) / np.sqrt(P * (1 - P) * (1. / n1 + 1. / n2))
'%.4f' % (1-stats.norm.cdf(Z))

'1.0000'

In [37]:
def proportions_diff_confint_ind(sample1, sample2, alpha = 0.05):    
    z = stats.norm.ppf(1 - alpha / 2.)
    
    p1 = float(sum(sample1)) / len(sample1)
    p2 = float(sum(sample2)) / len(sample2)
    
    left_boundary = (p1 - p2) - z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    right_boundary = (p1 - p2) + z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    
    return (left_boundary, right_boundary)

def proportions_diff_z_stat_ind(sample1, sample2):
    n1 = len(sample1)
    n2 = len(sample2)
    
    p1 = float(sum(sample1)) / n1
    p2 = float(sum(sample2)) / n2 
    P = float(p1*n1 + p2*n2) / (n1 + n2)
    
    return (p1 - p2) / np.sqrt(P * (1 - P) * (1. / n1 + 1. / n2))

def proportions_diff_z_test(z_stat, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    if alternative == 'two-sided':
        return 2 * (1 - stats.norm.cdf(np.abs(z_stat)))
    
    if alternative == 'less':
        return stats.norm.cdf(z_stat)

    if alternative == 'greater':
        return 1 - stats.norm.cdf(z_stat)

In [38]:
m = np.hstack((np.ones(203),np.zeros(718)))
w = np.hstack((np.ones(239),np.zeros(515)))

In [39]:
proportions_diff_confint_ind(m, w)

(-0.13922183141523897, -0.053905233215813156)

8.1534530895766011e-06

Проверьте гипотезу о равенстве долей любителей часто проводить вечера в баре среди мужчин и женщин. Посчитайте достигаемый уровень значимости, используя двустороннюю альтернативу. Введите номер первой значащей цифры (например, если вы получили $5.5\times10^{-8}$, нужно ввести 8).

In [74]:
proportions_diff_z_test(proportions_diff_z_stat_ind(m, w))

8.1534530895766011e-06

Посмотрим на данные General Social Survey 2014 года и проанализируем, как связаны ответы на вопросы "Счастливы ли вы?" и "Довольны ли вы вашим финансовым положением?"

                        |Не доволен |	Более или менее | Доволен |
                        |-----------|----------------_---|---------|
     Не очень счастлив  |	197	|         111	    |   33    |
    Достаточно счастлив |	382	|         685	    |   331   |
     Очень счастлив	 |    110    |         342	    |   333   |
     
Чему равно значение статистики хи-квадрат для этой таблицы сопряжённости? Округлите ответ до четырёх знаков после десятичной точки.

In [49]:
poll = np.array([[197, 111, 33], [382, 685, 331], [110, 342, 333]])

In [75]:
'%.4f' % stats.chi2_contingency(poll)[0]

'293.6831'

На данных из предыдущего вопроса посчитайте значение достигаемого уровня значимости. Введите номер первой значащей цифры (например, если вы получили $5.5\times10^{-8}$
 , нужно ввести 8).

In [53]:
chi2 = stats.chi2_contingency(poll)[1]
'%e' % chi2

'2.496430e-62'

Чему в предыдущей задаче равно значение коэффициента V Крамера для рассматриваемых признаков? Округлите ответ до четырёх знаков после десятичной точки.

V = $ \sqrt{\frac{χ2 (X1^n, X2^n)} {n (min (K1, K2) − 1)}} $

In [56]:
np.sqrt(chi2 / (poll.sum()*(min(poll.shape)-1)))

2.2238219864537978e-33

In [73]:
def V_stat(confusion_matrix):
    chi2 = stats.chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum()
    return np.sqrt(chi2 / (n*(min(confusion_matrix.shape)-1)))


'%.4f' % V_stat(poll)

'0.2412'